### Contour Integration visualisation
In this example we demonstrate approximation of $(e^{(\Delta t L/2 )}- 1)/L$, using complex contour integration

Here L is a diagonal operator arising from the spectral spatial discretisation. 

Unlike the matrix case, we can define a contour integration per point, and remove the analytic singularity. 

In the Matrix valued case, more careful consideration is required for the contour integration.

In [1]:
import os
os.environ["JAX_ENABLE_X64"] = "true"
import sys
sys.path.append('..')
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from ipywidgets import interact
from ml_collections import ConfigDict

from models.ETD_KT_CM_JAX_Vectorised import *
from filters import resamplers
from filters.filter import ParticleFilter
from jax import config
config.update("jax_enable_x64", True)

 The dictionary is specified in models/ETD_KT_CM_JAX_Vectorised.py, and loaded in as follows:

In [2]:
signal_params = ConfigDict(KDV_params_2)
print(signal_params)

Advection_basis_name: none
E: 1
Forcing_basis_name: none
P: 0
S: 0
c_0: 0
c_1: 1
c_2: 0.0
c_3: 2.0e-05
c_4: 0.0
dt: 0.001
equation_name: KdV
initial_condition: gaussian
method: Dealiased_SETDRK4
noise_magnitude: 0.0
nt: 4000
nx: 256
tmax: 4
xmax: 1
xmin: 0



Now we specify the model (refered to as a signal model), by instantiating the class.

We update the parameters and then rederive the dependent parameters

In [3]:
signal_model = ETD_KT_CM_JAX_Vectorised(signal_params)
initial_signal = initial_condition(signal_model.x, signal_params.E, signal_params.initial_condition)
signal_model.params.dt = signal_model.params.dt*0.1
signal_model.params.nt = signal_model.params.nt*10
signal_model.update_params(signal_model.params)# it is neccessary to update the derived parameters in the model, such as time dependent parameters
print(signal_params)

Advection_basis_name: none
E: 1
Forcing_basis_name: none
P: 0
S: 0
c_0: 0
c_1: 1
c_2: 0.0
c_3: 2.0e-05
c_4: 0.0
dt: 0.0001
equation_name: KdV
initial_condition: gaussian
method: Dealiased_SETDRK4
noise_magnitude: 0.0
nt: 40000
nx: 256
tmax: 4
xmax: 1
xmin: 0



To run the model outputting all timesteps we run the signal model, with nmax timesteps.

In [4]:
key = jax.random.PRNGKey(0)
final, all = signal_model.run(initial_signal, signal_model.params.nt, None, key)
print(signal_model.params)


Advection_basis_name: none
E: 1
Forcing_basis_name: none
P: 0
S: 0
c_0: 0
c_1: 1
c_2: 0.0
c_3: 2.0e-05
c_4: 0.0
dt: 0.0001
equation_name: KdV
initial_condition: gaussian
method: Dealiased_SETDRK4
noise_magnitude: 0.0
nt: 40000
nx: 256
tmax: 4
xmax: 1
xmin: 0



Here we note that we have L diagonal, such that contour integration can be done on the points. 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

dt = signal_model.params.dt
A1 = signal_model.A1
A2 = signal_model.A2
L = signal_model.L
g = signal_model.g
k = signal_model.k
L = L

R  =  1
M  =  32
r  =  R * jnp.exp(2j * jnp.pi * (jnp.arange(1, M + 1) - 0.5) / M) # 1/M,...2M-1/M,
LR =  dt * L[:, None] + r[None, :] #
Q  =  dt * jnp.mean( (jnp.exp(LR / 2) - 1) / LR, axis=-1)
# per point in the operator L, we center a unit circle of radius R, and then compute the integral over the circle using the trapezium rule.


plt.figure(figsize=(6, 6),dpi=300)
#plt.plot(ans_0.real, ans_0.imag, 'r--', label='exp(dt L/2 - 1)/L at zero') 
plt.scatter(LR.real.flatten(), LR.imag.flatten(), label='LR (dt*L + r)', marker='.',c='b')
plt.scatter((L*dt).real, (L*dt).imag, label='L', marker='.',c='k')
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Exponential of L in the Complex Plane')
plt.grid(True)
plt.axis('equal')
plt.legend(fontsize=12, loc='best', frameon=True)
plt.tight_layout()
plt.show()


# Plot the contour (circle) around several points LR[idx] for a range of indices
indices = [0, 2, 5, 10]  # you can choose any indices you want

def plot_contour(idx):
    center = L[idx] * dt
    contour_points = LR[idx]

    re = jnp.linspace(center.real - 2*R, center.real + 2*R, 100)
    im = jnp.linspace(center.imag - 2*R, center.imag + 2*R, 100)
    Re, Im = jnp.meshgrid(re, im)
    Z = Re + 1j * Im
    F = (jnp.exp(Z / 2) - 1) / Z

    fig = plt.figure(figsize=(10, 7), dpi=200)
    ax = fig.add_subplot(111, projection='3d')

    # Plot the center point as a red star at its value of |F(center)|
    ax.scatter(center.real, center.imag, abs(1/dt*Q[idx]), 
                   color='k', marker='o', s=200, label='Q (contour approx)')
    ax.scatter(center.real, center.imag, jnp.abs((jnp.exp(center / 2) - 1) / center), 
               color='red', marker='*', s=200, label='center F(center)')
    ax.plot_surface(Re, Im, jnp.abs(F), cmap='viridis', alpha=0.5, linewidth=0, antialiased=False)
    ax.plot(contour_points.real, contour_points.imag, jnp.zeros_like(contour_points.real), 
            'ro-', markersize=6, alpha=0.5)
    ax.plot(contour_points.real, contour_points.imag, jnp.abs((jnp.exp(contour_points / 2) - 1) / contour_points), 
            'r.-', label='Contour', markersize=6)
    ax.set_xlabel('Real(z)')
    ax.set_ylabel('Imag(z)')
    ax.set_zlabel(r'$| (e^{z/2}-1)/z |$')
    ax.set_title(f'Surface plot for index {idx}')
    ax.legend(fontsize=12, loc='best', frameon=True)
    plt.tight_layout()
    plt.show()

interact(plot_contour, idx=(0, L.shape[0]-1))


interactive(children=(IntSlider(value=127, description='idx', max=255), Output()), _dom_classes=('widget-inter…

<function __main__.plot_contour(idx)>

In [11]:
def method_one(dt,L):
    # Compute exp(dt*L/2) - 1 stably
    ans_0 = (jnp.exp(dt*L/2) - 1)/L

    return ans_0
# Compute (exp(dt*L/2) - 1) / L stably
#jnp.expm1(L/2*dt) / (L * dt) = (dt L/2 + dt^2 L^2/8 + dt^3 L^3/48 + ... -1)/ (L * dt) = 
def expm1_div_x(dt,L):
    # Use jnp.where to avoid division by zero
    return dt * jnp.where(jnp.abs(L/2*dt) > 1e-15, (jnp.expm1(L/2*dt)) / (L * dt), 1.0)

def taylor_exp(dt, L):
    # Compute the Taylor series expansion of exp(dt*L/2) - 1
    ans = dt * L / 2 + (dt * L / 2)**2 / 2 + (dt * L / 2)**3 / 6 
    return (ans-1) / (L*dt)

def stable_exp_minus1_over_z(z, threshold=1e-3):
    """
    Compute e^{z - 1} / z in a numerically stable way for small z
    """
    # Use a Taylor series for small z
    def taylor_approx(z):
        # up to z^4 is usually enough
        return (1/z + 1 + z/2 + z**2/6 + z**3/24) * jnp.exp(-1)

    # Use original expression otherwise
    def normal_expr(z):
        return jnp.exp(z - 1) / z

    return jnp.where(jnp.abs(z) < threshold, taylor_approx(z), normal_expr(z))

ans = expm1_div_x( dt , L)
ans_0 = method_one(dt, L)# unsuitable due to zero division.
ans2 = Q
ans3 = taylor_exp(dt, L)
diff_idx = jnp.argmax(jnp.abs(ans - ans2))
plt.scatter(ans2[diff_idx].real, ans2[diff_idx].imag, label='diff_index',c=k[diff_idx], marker='*', s=200)
#plt.scatter(ans_0.real, ans_0.imag, label='exp(dt L/2 - 1)/L at zero no handling', marker='x')
plt.scatter(ans.real, ans.imag, label=f'Manual error handled $(exp(dt L/2)-1)/L$',marker='o')
plt.scatter(ans2.real, ans2.imag, label='Q', marker='x')
#plt.scatter(ans3.real, ans3.imag, label='Taylor expansion', marker='s')

plt.legend()
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Comparison of expm1_div_x, Q, and method_one')
plt.show()

# ans2 is Q, which is computed using complex contour integration to approximate (exp(dt*L/2) - 1)/L stably for all L.
# This avoids the division by zero and numerical instability present in ans_0 (method_one).

diff_idx = jnp.argmax(jnp.abs(ans - ans2))
print("Index of largest difference:", diff_idx)
print("ans[{}] = {}, ans_0[{}] = {}".format(diff_idx, ans[diff_idx], diff_idx, ans_0[diff_idx]))
print("ans2 (Q) =", ans2[diff_idx])
print(L[0])
# Compute the L2 norm of the difference between Q (ans2) and ans

l2_diff = jnp.linalg.norm(ans2 - ans)
print("L2 norm of the difference between Q and ans:", l2_diff)
# Compute the L2 norm of the difference between Q (ans2) and ans, excluding diff_idx
mask = jnp.arange(ans.size) != diff_idx
l2_diff_excl = jnp.linalg.norm(ans2[mask] - ans[mask])
print("L2 norm of the difference between Q and ans (excluding diff_idx):", l2_diff_excl)


Index of largest difference: 0
ans[0] = (0.0001+0j), ans_0[0] = (nan+nanj)
ans2 (Q) = (5.0000000000000016e-05+1.6046192152785466e-21j)
0j
L2 norm of the difference between Q and ans: 4.999999999999999e-05
L2 norm of the difference between Q and ans (excluding diff_idx): 1.3358204864773036e-19


This results in a (nt, E, nx) sized array, nt denotes the number of timesteps, E denotes the number of ensemble members, nx denotes the number of spatial points.

In [7]:
print(all.shape) # particles

(40000, 1, 256)


Here we insert the initial condition into the outputted solution. 

In [8]:
print(initial_signal.shape)
print(all.shape)
signal = jnp.concatenate([initial_signal[None,...], all], axis=0)
print(signal.shape)  # (nt+1, n_particles, n_states)

(1, 256)
(40000, 1, 256)
(40001, 1, 256)


We plot an interactive plot

In [12]:
def plot(time_index):
    plt.figure(figsize=(8, 4))
    plt.plot(signal_model.x, signal[time_index, 0, :], color='k', lw=2)
    plt.xlabel('Spatial coordinate $x$', fontsize=14)
    plt.ylabel('Signal $u(x)$', fontsize=14)
    plt.title(f'KdV at $t={time_index * signal_model.params.dt:.2f}$', fontsize=16)
    plt.grid(True, which='both', linestyle='--', alpha=0.6)
    plt.xlim(signal_model.x[0], signal_model.x[-1])
    plt.tight_layout()
    plt.show()

interact(plot, time_index=(0, signal_model.params.nt))

interactive(children=(IntSlider(value=20000, description='time_index', max=40000), Output()), _dom_classes=('w…

<function __main__.plot(time_index)>

In [10]:
plt.figure(figsize=(10, 6))
plt.imshow(all[:, 0, :], aspect='auto', origin='lower', cmap='viridis')
plt.colorbar(label='Signal amplitude')
plt.title('Space-Time Evolution of the Signal', fontsize=16)
plt.xlabel('Spatial Index (nx)', fontsize=12)
plt.ylabel('Time Step (nt)', fontsize=12)
plt.tight_layout()
plt.show()
